## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/AIML/Project_DogBreedClassification/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

In [8]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import os
import cv2
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm
from sklearn.metrics import log_loss, accuracy_score, recall_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


import keras
from keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
#from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [0]:
#start = dt.datetime.now()

### Read labels.csv file using pandas

In [0]:
labels = pd.read_csv('./labels.csv')

In [10]:
labels.shape

(10222, 2)

In [11]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10222 entries, 0 to 10221
Data columns (total 2 columns):
id       10222 non-null object
breed    10222 non-null object
dtypes: object(2)
memory usage: 159.8+ KB


In [12]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


### Print the count of each category of Dogs given in the dataset



In [13]:
labels.breed.value_counts()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
golden_retriever         67
komondor                 67
brabancon_griffon        67
eskimo_dog               66
briard                   66
Name: breed, Length: 120, dtype: int64

### Get one-hot encodings of labels

In [0]:
breed_ohl=np.array(pd.get_dummies(labels.breed))

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
INPUT_SIZE = 128
NUM_CLASSES = 16
SEED = 1987

In [15]:
labels = pd.read_csv('/content/labels.csv')
sample_submission = pd.read_csv('/content/sample_submission.csv')
print(len(listdir('/content/train')), len(labels))
print(len(listdir('/content/test')), len(sample_submission))

10222 10222
10357 10357


In [16]:
selected_breed_list = list(labels.groupby('breed').count().sort_values(by='id', ascending=False).head(NUM_CLASSES).index)
labels = labels[labels['breed'].isin(selected_breed_list)]
labels['target'] = 1

group = labels.groupby(by='breed', as_index=False).agg({'id': pd.Series.nunique})
group = group.sort_values('id',ascending=False)
print(group)
labels['rank'] = group['breed']

#labels['rank'] = labels.groupby('breed').rank()['id']
labels_pivot = labels.pivot('id', 'breed', 'target').reset_index().fillna(0)
np.random.seed(seed=SEED)
rnd = np.random.random(len(labels))
train_idx = rnd < 0.8
valid_idx = rnd >= 0.8
y_train = labels_pivot[selected_breed_list].values
ytr = y_train[train_idx]
yv = y_train[valid_idx]

                   breed   id
13    scottish_deerhound  126
10           maltese_dog  117
0           afghan_hound  116
6            entlebucher  115
4   bernese_mountain_dog  114
14              shih-tzu  112
7         great_pyrenees  111
11            pomeranian  111
2                basenji  110
12               samoyed  109
1               airedale  107
15       tibetan_terrier  107
5                  cairn  106
9               leonberg  106
3                 beagle  105
8       japanese_spaniel  105


In [0]:
def read_img(img_id, train_or_test, size):
    """Read and resize image.
    # Arguments
        img_id: string
        train_or_test: string 'train' or 'test'.
        size: resize the original image.
    # Returns
        Image as numpy array.
    """
    img = image.load_img(join('/content/', train_or_test, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    return img

In [18]:
POOLING = 'avg'
x_train = np.zeros((len(labels), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, img_id in tqdm(enumerate(labels['id'])):
    img = read_img(img_id, 'train', (INPUT_SIZE, INPUT_SIZE))
    x = preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_train[i] = x
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

1777it [00:05, 320.55it/s]

Train Images shape: (1777, 128, 128, 3) size: 87,343,104


In [19]:
Xtr = x_train[train_idx]
Xv = x_train[valid_idx]
print((Xtr.shape, Xv.shape, ytr.shape, yv.shape))

((1409, 128, 128, 3), (368, 128, 128, 3), (1409, 16), (368, 16))


Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
#Done Above

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
#Done above

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
test_img=sample_submission.id.values

In [21]:
sample_submission.id.nunique()

10357

Run the below code to load the test image files in x_test_feature

In [0]:
#import cv2
x_test_feature = []
i = 0 # initialisation
for f in tqdm(test_img): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f), 0)
    img_resize = cv2.resize(img, (200, 200)) 
    img_resize=(img_resize - img_resize.mean()) / img_resize.std() # normalized images
    x_test_feature.append(img_resize)

Normalize the test data and convert it into 4 dimensions

In [0]:
#Done above step

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [26]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5),activation='relu',input_shape=(128,128,3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(16, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 124, 124, 32)      2432      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 122, 122, 32)      9248      
_________________________________________________________________
flatten_3 (Flatten)          (None, 476288)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               121929984 
_________________________________________________________________
dense_6 (Dense)              (None, 16)                4112      
Total params: 121,945,776
Trainable params: 121,945,776
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
optimizer=keras.optimizers.Adadelta(),
metrics=['accuracy'])

### Use batch_size = 128 and epochs = 10 and execute the model

In [28]:
history = model.fit(Xtr, ytr,
batch_size=128,
epochs=10,
verbose=1,
validation_data=(Xv, yv))
score = model.evaluate(Xv, yv, verbose=0)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 1409 samples, validate on 368 samples
Epoch 1/10





1409/1409 [==============================] - 18s 13ms/step - loss: 14.8325 - acc: 0.0710 - val_loss: 15.1983 - val_acc: 0.0571
Epoch 2/10
1409/1409 [==============================] - 1s 1ms/step - loss: 15.0771 - acc: 0.0646 - val_loss: 15.1983 - val_acc: 0.0571
Epoch 3/10
1409/1409 [==============================] - 1s 1ms/step - loss: 15.0771 - acc: 0.0646 - val_loss: 15.1983 - val_acc: 0.0571
Epoch 4/10
1409/1409 [==============================] - 2s 1ms/step - loss: 15.0771 - acc: 0.0646 - val_loss: 15.1983 - val_acc: 0.0571
Epoch 5/10
1409/1409 [==============================] - 2s 1ms/step - loss: 15.0771 - acc: 0.0646 - val_loss: 15.1983 - val_acc: 0.0571
Epoch 6/10
1409/1409 [==============================] - 2s 1ms/step - loss: 15.0771 - acc: 0.0646 - val_loss: 15.1983 - val_acc: 0.0571
Epoch 7/10
1409/1409 [===========

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:

train_generator = train_datagen.flow(Xtr, ytr, batch_size=128)

validation_generator = test_datagen.flow(Xv, yv, batch_size=128)

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
optimizer=keras.optimizers.Adadelta(),
metrics=['accuracy'])

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [32]:
model.fit_generator(
        train_generator,
        epochs=1,
        validation_data=validation_generator)

##Executing only for 1 epoch since going out of RAM

Epoch 1/1
12/12 [==============================] - 6s 533ms/step - loss: 6.5438 - acc: 0.0642 - val_loss: 2.7957 - val_acc: 0.0842


# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [33]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

In [34]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [39]:
base_model.trainable = False
x = (base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(16, activation = 'softmax')(x)
top_model = Model(input = base_model.input, output = predictions)
top_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

top_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


### Make all the layers in the base_model (VGG16) to be non-trainable

In [40]:
#for layer in base_model.layers:
#    layer.trainable = False

# Freeze the layers except the last 4 layers
for layer in top_model.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in top_model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f09303aa898> False
<keras.layers.convolutional.Conv2D object at 0x7f09447ffef0> False
<keras.layers.convolutional.Conv2D object at 0x7f09447ffc88> False
<keras.layers.pooling.MaxPooling2D object at 0x7f093040ac88> False
<keras.layers.convolutional.Conv2D object at 0x7f09303b28d0> False
<keras.layers.convolutional.Conv2D object at 0x7f09303bc860> False
<keras.layers.pooling.MaxPooling2D object at 0x7f09303c25c0> False
<keras.layers.convolutional.Conv2D object at 0x7f09303c9f98> False
<keras.layers.convolutional.Conv2D object at 0x7f09303cde10> False
<keras.layers.convolutional.Conv2D object at 0x7f09303d5cf8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f098e346320> False
<keras.layers.convolutional.Conv2D object at 0x7f098e34ec18> False
<keras.layers.convolutional.Conv2D object at 0x7f098e354cf8> False
<keras.layers.convolutional.Conv2D object at 0x7f098e35cbe0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f098e3693c8>

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

In [0]:
top_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['acc'])

In [49]:
history = top_model.fit(Xtr, ytr,
batch_size=128,
epochs=15,
verbose=1,
validation_data=(Xv, yv))
score = top_model.evaluate(Xv, yv, verbose=0)

Train on 1409 samples, validate on 368 samples
Epoch 1/15
1409/1409 [==============================] - 1s 1ms/step - loss: 2.9578e-04 - acc: 1.0000 - val_loss: 0.9068 - val_acc: 0.8179
Epoch 2/15
1409/1409 [==============================] - 1s 1ms/step - loss: 2.7592e-04 - acc: 1.0000 - val_loss: 0.9158 - val_acc: 0.8179
Epoch 3/15
1409/1409 [==============================] - 1s 1ms/step - loss: 2.6604e-04 - acc: 1.0000 - val_loss: 0.9352 - val_acc: 0.8179
Epoch 4/15
1409/1409 [==============================] - 1s 1ms/step - loss: 2.5152e-04 - acc: 1.0000 - val_loss: 0.9246 - val_acc: 0.8179
Epoch 5/15
1409/1409 [==============================] - 1s 1ms/step - loss: 2.3698e-04 - acc: 1.0000 - val_loss: 0.9319 - val_acc: 0.8207
Epoch 6/15
1409/1409 [==============================] - 1s 1ms/step - loss: 2.2149e-04 - acc: 1.0000 - val_loss: 0.9360 - val_acc: 0.8179
Epoch 7/15
1409/1409 [==============================] - 1s 1ms/step - loss: 2.0890e-04 - acc: 1.0000 - val_loss: 0.9332 - val

Try to get training and validation accuracy to be more than 90%

In [44]:
vgg_bottleneck = VGG16(weights='imagenet', include_top=False, pooling=POOLING)
train_vgg_bf = vgg_bottleneck.predict(Xtr, batch_size=32, verbose=1)
valid_vgg_bf = vgg_bottleneck.predict(Xv, batch_size=32, verbose=1)
print('VGG train bottleneck features shape: {} size: {:,}'.format(train_vgg_bf.shape, train_vgg_bf.size))
print('VGG valid bottleneck features shape: {} size: {:,}'.format(valid_vgg_bf.shape, valid_vgg_bf.size))

368/368 [==============================] - 0s 892us/step
VGG train bottleneck features shape: (1409, 512) size: 721,408
VGG valid bottleneck features shape: (368, 512) size: 188,416


In [0]:
##LogReg on VGG bottleneck features

In [46]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED)
logreg.fit(train_vgg_bf, (ytr * range(NUM_CLASSES)).sum(axis=1))
valid_probs = logreg.predict_proba(valid_vgg_bf)
valid_preds = logreg.predict(valid_vgg_bf)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [47]:
print('Validation VGG LogLoss {}'.format(log_loss(yv, valid_probs)))
print('Validation VGG Accuracy {}'.format(accuracy_score((yv * range(NUM_CLASSES)).sum(axis=1), valid_preds)))

Validation VGG LogLoss 1.20023391125592
Validation VGG Accuracy 0.7961956521739131


In [0]:
## Extract Xception bottleneck features

In [51]:
INPUT_SIZE = 299
POOLING = 'avg'
x_train = np.zeros((len(labels), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, img_id in tqdm(enumerate(labels['id'])):
    img = read_img(img_id, 'train', (INPUT_SIZE, INPUT_SIZE))
    x = xception.preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_train[i] = x
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

1777it [00:06, 268.18it/s]

Train Images shape: (1777, 299, 299, 3) size: 476,596,731


In [52]:
Xtr = x_train[train_idx]
Xv = x_train[valid_idx]
print((Xtr.shape, Xv.shape, ytr.shape, yv.shape))
xception_bottleneck = xception.Xception(weights='imagenet', include_top=False, pooling=POOLING)
train_x_bf = xception_bottleneck.predict(Xtr, batch_size=32, verbose=1)
valid_x_bf = xception_bottleneck.predict(Xv, batch_size=32, verbose=1)
print('Xception train bottleneck features shape: {} size: {:,}'.format(train_x_bf.shape, train_x_bf.size))
print('Xception valid bottleneck features shape: {} size: {:,}'.format(valid_x_bf.shape, valid_x_bf.size))

((1409, 299, 299, 3), (368, 299, 299, 3), (1409, 16), (368, 16))


368/368 [==============================] - 2s 6ms/step
Xception train bottleneck features shape: (1409, 2048) size: 2,885,632
Xception valid bottleneck features shape: (368, 2048) size: 753,664


In [53]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED)
logreg.fit(train_x_bf, (ytr * range(NUM_CLASSES)).sum(axis=1))
valid_probs = logreg.predict_proba(valid_x_bf)
valid_preds = logreg.predict(valid_x_bf)
print('Validation Xception LogLoss {}'.format(log_loss(yv, valid_probs)))
print('Validation Xception Accuracy {}'.format(accuracy_score((yv * range(NUM_CLASSES)).sum(axis=1), valid_preds)))

Validation Xception LogLoss 0.06823319196427392
Validation Xception Accuracy 0.9809782608695652


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [0]:
# 98% accuracy 0.06 LogLoss.